# Powtórka
wyprowadziliśmy metodę uczenia wielowarstwowych sieci nieliniowych bazującą na minimalizacji funkcji kosztu metodą gradientową. W dzisiejszym wykładzie zastanowimy się jak robić to w sposób:
* szybszy
* zmniejszający ryzyko wpadania w minima lokalne
* prowadzący do lepszej generalizacji

# Przyspieszanie uczenia 
## Bezwładność 
![Przykładowa ewolucja wag na tle konturu funkcji kosztu. Lewy ślad: brak bezwładności, prawy z włączoną bezwładnością](https://brain.fuw.edu.pl/edu/images/0/08/Bezwładnosc.png)

Podobnie jak dla sieci liniowej można uzyskać poprzez dodanie członu bezwładności do formuły zmiany wag:

$\qquad$ $\Delta w^{(j+1)} = - \alpha_1 \frac{\partial J}{\partial w} + \alpha_2\Delta w^{(j)}  $

* Dla prawie płaskiej powierzchni kosztu efektywny współczynnik uczenia jest
$ \frac{1}{1-\alpha_2}$ razy większy niż w przypadku algorytmu bez bezwładności. 

* Dla typowych wartości $\alpha_1 = 0.1$ i $\alpha_2 = 0.9$ otrzymujemy około 10 krotne przyspieszenie!

## Adaptacyjny dobór parametrów 
Kiedy obserwujemy proces minimalizacji funkcji błędu to często widać, że stała wartość parametru uczenia $\alpha_1$ nie jest optymalna. 
* Czasami widać, że funkcja długi czas maleje prawie monotonicznie - wtedy lepiej byłoby mieć większą wartość  $\alpha_1$,
* kiedy indziej bardzo oscyluje - to świadczy o zbyt dużej wartości $\alpha_1$. 

### Prosty pomysł 
Jednym z prostych pomysłów na zautomatyzowanie procesu dobierania wartości $\alpha_1$ jest następujący schemat zmiany tego parametru:

$\qquad$ $
\Delta \alpha_1  = \left\{ 
\begin{array}{lcl}
+ a& \text{dla} & \Delta J <0 \\
-b \alpha_1 & \text{dla} & \Delta J >0 \\
0 & \text{dla pozostalych przypadkow}
\end{array}
\right.
$

Po wykonaniu kroku prowadzącego do zwiększenia funkcji kosztu warto go wycofać.

## Elastyczna wsteczna propagacja błędu 
Resilient Backpropagation — do zmiany wag wykorzystujemy jedynie informację o znaku gradientu. W poniższych formułach $w_q^p$ to waga łącząca wyjście neuronu $q$ z odpowiednim (q-tym) wejściem neuronu $p$.

$\qquad$ $
\Delta w_q^{p(j)}  = - \text{sign} \left(\frac{\partial J}{\partial w_p^q}\right) \Delta_q^{p(j)} 
$
gdzie: $J$ jest miarą błędu po prezentacji wszystkich bodźców, zaś $\Delta_q^{p(j)}$:
$\qquad$ $
\Delta_q^{p(j)}  = \left\{ 
\begin{array}{lcl}
\eta^+ \Delta_q^{p(j-1)} & \text{dla} & \frac{\partial J^{(j-1)}}{\partial w_p^q}\frac{\partial J^{(j)}}{\partial w_p^q} >0 \\
\eta^- \Delta_q^{p(j-1)} & \text{dla} & \frac{\partial J^{(j-1)}}{\partial w_p^q}\frac{\partial J^{(j)}}{\partial w_p^q} <0 \\
\Delta_q^{p(j-1)} & \text{dla pozostalych przypadkow}
\end{array}
\right.
$

gdzie: $ 0 < \eta^- < 1 < \eta^+$. 

Dodatkowo ustala się ograniczenie na możliwe wartości $ \Delta_{min} < \Delta_p^q < \Delta_{max}$. 

Standardowo $\Delta_{min} \approx 10^{-6}$ a $ \Delta_{max} \approx 50$. 

Dla sigmoidalnych funkcji odpowiedzi metoda ta może poprawić uczenie w obszarze ogonów sigmoidy, gdzie wartość gradientu jest bardzo mała.

## Metoda najszybszego spadku 
Kolejnej wartości wagi szukamy wzdłuż prostej wyznaczonej przez poprzedni wektor wag $w^{(j)}$ i kierunek $d^{(j)}$, zmieniając $\lambda$ tak, aby zminimalizować w danym kierunku $J$:

$\qquad$ $w^{(j+1)} = w^{(j)} + \lambda d^{(j)}$

Kierunek $d$ wybieramy przeciwnie do gradientu $J$

$\qquad$ $ d^{(j)} = -\nabla J(w^{(j)})$

Zauważmy, że stary i nowy kierunek minimalizacji są ortogonalne, bo $\lambda$ jest dobrana tak, aby minimalizować funkcję kosztu w kierunku $d^{(j)}$ tzn.:

$\qquad$ $\frac{\partial}{\partial \lambda} J(w^{(j)} + \lambda d^{(j)}) = 0 $

Ale rozwijając powyższy wzór widzimy, że:


$\qquad$ $\frac{\partial}{\partial \lambda} J(w^{(j)} + \lambda d^{(j)}) =  \frac{\partial J\left(w^{(j+1)}\right)}{\partial w} \frac{\partial (w^{(j)}+ \lambda d^{(j)})}{\partial\lambda}  = \nabla J(w^{(j+1)}) \cdot d^{(j)} = -d^{(j+1)} \cdot d^{(j)} =0
$

## Metoda gradientu sprzężonego
Poprzednią metodę można udoskonalić przez rezygnację z ortogonalności kolejnych kroków:

$\qquad$ $d^{(j+1)} = -\nabla J \left( w^{(j+1)} \right) + \beta d^{(j)}$

i trzeba chytrze dobrać $\beta$ tak, aby jak najmniej psuć efekt osiągnięty w poprzednim kroku. Nowy kierunek szukania powinien więc być taki, aby z dokładnością pierwszego rzędu nie zmieniał składowej gradientu, która w poprzednim kroku została wyzerowana. A zatem chcemy, aby z dokładnością do wyrazów pierwszego stopnia, spełnione było:

$\qquad$ $ d^{(j)} \cdot \nabla J\left( w^{(j)} + \lambda d^{(j+1)} \right) = 0 $

praktyczny sposób na znalezienie $ \beta$ spełniającego powyższy warunek podaje reguła Polaka-Ribiere‘a: 

$\qquad$ $ \beta = \frac{\left( \nabla J\left (w^{(j+1)} \right)  - \nabla J \left( w^{(j)}\right)\right) \cdot \nabla J\left(w^{(j+1)}\right)}{
\left( \nabla J \left(w^{(j)}\right)\right)^2}$

## Metody quasi-Newtona

Oryginalna metoda Newtona polega na minimalizacji funkcji kosztu z wykorzystaniem drugich pochodnych.
Rozwijając $J(w)$ wokół bieżącego wektora wag $w_0$ mamy: 

$\qquad$ $J(w)=J(w_0)+(w-w_0)\cdot \nabla J\left( w_0\right) + \frac{1}{2} (w - w_0) \cdot H \cdot(w - w_0) +... $ (∗)

gdzie: H to ''hesjan'' (macierz drugich pochodnych cząstkowych $ H_{ij} =\frac{\partial^2 J}{\partial w_i \partial w_j}$ ).

Różniczkowanie (* * *) daje: 

$\qquad$ $ \nabla J(w) = \nabla J(w_0) + H \cdot (w - w_0) + \dots$

chcemy znaleźć minimum $J$ czyli spełnić warunek $ \nabla J(w) = 0$:

$\qquad$ $ \nabla J(w_0) + H \cdot (w - w_0) = 0$

stąd: 

$\qquad$ $w  = w_0 + H^{-1} \nabla J \left(w_0 \right) $

Wzór ten można stosować iteracyjnie.

Metoda w oryginalnej postaci jest bardzo kosztowna obliczeniowo ($O(n^3)$) i jest niestabilna numerycznie. Stąd też realne implementacje są nieco inne i zasadniczo polegają na iteracyjnej aktualizacji hesjanu. Zwykle wymaga mniej kroków niż metoda gradientów sprzężonych, ale w każdym kroku jest więcej obliczeń i trzeba mieć pamięć na przechowywanie hesjanu.

## Algorytm Lavenberg-Marquardt'a
Metoda ta korzysta z faktu, że hesjan może być przybliżony przez:

$\qquad$ $ \mathbf{H} \approx \mathbf{J}^T \mathbf{J}$

gdzie: $\mathbf{J}$ — macierz jakobiego (macierz pierwszych pochodnych cząskowych) natomiast

$\qquad$ $\nabla J \left(w^{(j)}\right) = \mathbf{J}^T (z^{(j)} - y^{(j)})$

W metodzie tej wagi zmieniamy:

$\qquad$ $w^{(j+1)} = w^{(j)} + [\mathbf{J}^T\mathbf{ J} + \mu \mathbf{I}]^{-1} \mathbf{J}^T (z^{(j)} - y^{(j)})$

dla $ \mu = 0$ jest to metoda Newtona z przybliżoną wartością hesjanu, dla $ \mu$ dużego metoda dąży do zwykłej metody gradientowej. Metoda Newtona jest szybsza i dokładniejsza w pobliżu minimum $J$.

$ \mu$ jest zmniejszane po każdym udanym kroku a zwiększane jeśli w danym kroku $J$ wzrosło.

## Podsumowanie metod przyspieszania uczenia

* Ogólnie: Za szybkość płacimy ilością koniecznej pamięci i wymaganą precyzją obliczeń
* Algorytm Lavenberg-Marquardt'a: jest najszybszym algorytmem w problemach aproksymacji funkcji dla sieci o średniej wielkości (do kilkuset wag). Prowadzi też, na ogół, w tych problemach do lepszego dopasowania w sensie błędu średniokwadratowego od pozostałych algorytmów. Jest jednak kosztowny jeśli chodzi o zapotrzebowanie na pamięć.
* Resilient Backpropagation: wydaje się być najlepszy w problemach rozpoznawania wzorców, ale nie nadaje się w zasadzie do aproksymacji funkcji. Nie jest pamięciożerny.
* Algorytm gradientów sprzężonych: jest najbardziej uniwersalnym algorytmem. Ma umiarkowane wymagania co do ilości pamięci.
* Zwykła metoda gradientowa: jest najwolniejsza, ale może to być użyteczne jeśli bardziej niż na czasie zależy nam na generalizacji.
* Nieliniowość typu ReLu jest odporna na problem wysycania gradientu.

# Problem minimów lokalnych 
Wszystkie metody minimalizacji funkcji kosztu mogą utknąć w minimach lokalnych. Kilka metod, które mogą pomóc zmniejszyć problemy z minimami lokalnymi:
* uniknięcie wysycenia sigmoid już na samym początku uczenia. Np. dla unormowanego wejścia i dla sigmoidy z $\beta = 1 $ wybranie początkowych wag losowych o takich wartościach, że średnie pobudzenie neuronu $a$ jest mniejsze, ale nie za bardzo niż 1 (można losować wagi rzędu $\sqrt \frac{1}{k_i}$	gdzie $k_i$ ilość wejść do jednostki $i$);
* poprawianie wag po każdej prezentacji wzorca, przy czym wzorce prezentowane są w losowej kolejności;
* zastosowanie jednostek stochastycznych — gradient oraz dodatkowy parametr T temperatura kontrolują prawdopodobieństwo zmiany wagi w określonym kierunku;
* delikatne losowe zmiany wag;
* przy każdej prezentacji wzorca dodawanie do niego troszkę szumu. Dodanie szumu zawsze spowolni proces uczenia, przy czym mała dawka może pomóc uniknąć minimów lokalnych, duża - znacznie spowalnia uczenie.

# Twierdzenie o potencjalnych możliwościach sieci
Sieć nieliniowa co najmniej dwuwarstwowa może aproksymować dowolną funkcję swoich wejść, ze z góry zadaną dokładnością. Konieczna jest jedynie dostatecznie duża ilość jednostek w warstwach ukrytych. Do aproksymacji dowolnej funkcji ciągłej wystarcza jedna warstwa ukryta. 

Uzasadnienie nieformalne:
>  Każda “rozsądna” funkcja $ F_i{X_k}$ może być przedstawiona jako liniowa kombinacja “wypukłości”, z których każda jest różna od zera tylko w pobliżu $X_k$.
> Takie “wypukłości” można skonstruować z dwóch warstw ukrytych. 

Gdzie jest problem?
* twierdzenie mówi jedynie o istnieniu rozwiązania 
* w ogólności nie wiadomo ile jednostek w warstwach stanowi “dostatecznie dużą ilość”
* nie ma gwarancji, że problem da się rozwiązać metodą wstecznej propagacji błędu, tzn. że dotrzemy do minimum globalnego funkcji kosztu.

> Trywialne twierdzenie: uczenie sieci zawsze się uda jeśli zastosujemy prawidłowy preprocesor.

Jeśli w problemie występują symetrie, to o ile to możliwe warto przenieść ich analizę do fazy preprocesingu, bo powodują one powstanie w funkcji kosztu okresowości, wielokrotnych minimów lokalnych, płaskich dolin i wyżyn.

Ze standardowych technik przygotowywania danych (nie tylko dla sieci nuropodobnych) warto rozważyć: 
* wyskalowanie danych, 
* normalizację danych, 
* przeprowadzenie analizy składowych głównych.

# Generalizacja
## O co tu chodzi? 
![Schemat ilustrujący koncepcję generalizacji](https://brain.fuw.edu.pl/edu/images/f/f7/Generalizacja.png)

Na rysunku obok przedstawiona jest schematycznie koncepcja generalizacji. 

Wyobraźmy sobie, że jest pewna przestrzeń P, która zawiera pary, np. liczb {((a,b), c)}. 

W tym przykładzie jest to przestrzeń wszystkich odwzorowań $\mathcal{R}^2 \rightarrow \mathcal{R}$. 

Niektóre z tych par reprezentują pewną konkretną relację R: np. są to pary spełniające warunek $c=\sqrt{a^2 +b^2}$.

Wyobraźmy sobie dalej, że mamy dane dwa skończone zestawy par, które tą relację spełniają, ale oczywiście nie są w stanie obejmować wszystkich możliwych par. 

Jeden z nich oznaczymy U, a drugi T. Załóżmy, że mamy dwie wersje sieci, które uczymy na zbiorze U (mogą się one różnić architekturą, albo punktem startu procedury uczącej, albo ilością iteracji algorytmu uczącego itp.). 

Po procesie uczenia sieci te mają mały i porównywalny błąd na zbiorze U, ale jedna z nich nauczyła się relacji wskazanej na rys. jako g1 a druga relacji g2. 

Na podstawie rezultatów odtwarzania przykładów ze zbioru testowego mówimy, że sieć druga ma lepszą generalizację niż sieć pierwsza.

## Jakie mogą być przyczyny złej generalizacji?

* Architektura nie wystarczająca do reprezentacji relacji

* Architektura zbyt złożona ... 
... do danego odwzorowania i zbyt długi proces uczenia:

* Dotyczy to sytuacji, gdy zbiór uczący zawiera przykłady danej relacji z szumem. Sieć ma tak dużo parametrów, że jest w stanie nauczyć się szczegółów zbioru uczącego nie związanych z interesującą nas relacją. Zjawisko takie nazywamy ''przeuczeniem''.

## Uczenie z kryterium wczesnego stopu 
*  Oprócz zbioru uczącego posiadamy rozłączny z nim zbiór testowy.
* W trakcie uczenia obliczamy błąd popełniany przez sieć na zbiorze uczącym i na zbiorze testowym. 
* W większości przypadków powinniśmy w początkowej fazie uczenia obserwować malenie błędu na obu zbiorach. 
* Oczekujemy, że w pewnym momencie, kiedy sieć zaczyna się uczyć zbędnych szczegółów to błąd na zbiorze uczącym będzie nadal malał, a na zbiorze testowym zacznie rosnąć. 
* Uczenie prowadzimy tylko do momentu, kiedy błąd na obu zbiorach maleje. 

<img src="https://brain.fuw.edu.pl/edu/images/7/76/Wczesny_stop.png", width = 1600>

## Optymalizacja architektury: Regularyzacja
> Pomysł: zacznijmy od dużej sieci i po pewnym cyklu uczenia przeanalizujmy połączenia w sieci usuwając mało istotne połączenia lub jednostki. Następnie powtórzmy uczenie.

Można spróbować tak zmodyfikować regułę zmiany wag, aby połączenia nieistotne same dążyły do 0; po standardowym kroku uczenia zmniejszamy wagi: 

$\qquad$ $ w_q^{p(j+1)} = (1- \epsilon) w_q^{p(j)} $   (*)

Jest to równoważne modyfikacji funkcji kosztu:

$\qquad$ $J = J_0 + \frac{1}{2} \gamma \sum_{pq}\left(w_q^p \right)^2$

przy $\epsilon  = \alpha \gamma$.

Konsekwencje:
* rozwiązanie jest “gładsze”
* wygładzenie funkcji kosztu likwiduje część minimów lokalnych

Powyższa funkcja kosztu prowadzi do preferowania większej liczby małych wag zamiast jednej dużej. Sytuację poprawia wyrażenie:

$\qquad$ $ J = J_0 +\frac{1}{2}\gamma \sum_{pq} \frac{\left(w_q^p\right)^2}{1+\left(w_q^p \right)^2}$

co jest równoważne (*) przy 

$\qquad$ $\epsilon_q^p = \frac{\alpha \gamma}{\left( 1+ \left(w_q^p \right)^2\right)^2}$. 

Dzięki temu małe wagi zanikają szybciej niż duże. To załatwia problem zanikania niepotrzebnych połączeń. 

Aby zautomatyzować usuwanie zbędnych jednostek możemy zastosować:

$\qquad$ $ \epsilon^p = \frac{\alpha \gamma}{ \left( 1+\sum_q \left( w_q^p \right)^2\right)^2}$
dla wszystkich wejść do jednostki $p$. 

Powoduje to szybsze zanikanie wag dla jednostek, które mają małe wagi wejściowe.

## Diagnostyka i debugowanie algorytmu uczącego

Załóżmy, że zaimplementowaliśmy algorytm uczący na określonym (niewielkim) zbiorze danych. Mamy 20 % błędnych decyzji. Co z tym robić dalej?

Potencjalnie można:
* poprawić algorytm
* zdobyć więcej przykładów do ciągu uczącego
* wypróbować więcej cech lub ograniczyć przestrzeń cech
* pouczyć dłużej albo zmienić algorytm optymalizacyjny
* zmienić parametry uczenia (współczynnik szybkości, regularyzacji, itp.)
Każda z tych metod naprawia jakiś (ale inny) problem.

Aby zdiagnozować problem dobrze jest analizować wykresy:
* błędu od długości uczenia 
  * czy algorytm jest zbieżny? -> jeśli nie to możemy próbować poprawić to zmniejszając prędkość uczenia lub zmieniając procedurę optymalizacyjną
  

* błędu od rozmiaru zbioru uczącego
  * Jeśli błąd na zbiorze uczącym dużo niższy niż na testowym to możemy mieć problem przeuczenia
    * można: dodać przykładów, zmniejszyć rozmiar wejścia, włączyć regularyzację
    

  * Duży błąd na zbiorze uczącym i na testowym -> źle dobrane cechy lub struktura klasyfikatora 
    *  zmienić zestaw cech 
    * wzbogacić strukturę klasyfikatora (np. więcej jednostek ukrytych)

In [5]:
# -*- coding: utf-8 -*-
import numpy as np

def g(x):
    y = 1./(1+np.exp(-x))
    return y
    
def g_prim(x):
    y = x*(1-x)
    return y
    
    
#zbiór uczący:
# wejście, 
X = np.array([  [0,0],
                [0,1],
                [1,0],
                [1,1] ])
    
# wyjście            
Y = np.array([[0,1],
              [1,0],
              [1,0],
              [0,1]])
            
              
# definiujemy rozmiary sieci:
N_wej = X.shape[1] 
N_hid = 3
N_wyj = Y.shape[1]
                            
# inicjujemy połączenia
# wagi ułożone są tak, że w kolejnych wierszach są kolejne neurony 
# a w kolumnach wagi od konkretnego neuronu 
# to +1 jest wagą dla obciążenia
w_1 = 2*np.random.random((N_hid, N_wej+1)) - 1# pomiędzy warstwą pierwszą (wejściem) a warstwą ukrytą
w_2 = 2*np.random.random((N_wyj, N_hid+1)) - 1

                                                
for cykl in range(10000):
    bl =0
    D_1 = np.zeros((N_hid,N_wej+1))
    D_2 = np.zeros((N_wyj,N_hid+1))
    
    
    for i in range(0,4):
        # weźmy przykład i-ty
        
        x = X[i,:].reshape(X.shape[1],1)
        y = Y[i,:].reshape(Y.shape[1],1)
        
        # propagacja "w przód"
        a_0 = np.vstack((1,x))  # z warstwy wejściowej (zerowej) wychodzi a_0
        
        z_1 = np.dot( w_1, a_0 )# na warstwe 1 wchodzą iloczyny skalarne 
        a_1 = np.vstack((1,g(z_1))) # dokładamy 1 i dostaję wyjście z warstwy 1
        
        z_2 = np.dot( w_2, a_1 ) # na warstwe 3 wchodzą iloczyny skalarne 
        a_2 = g(z_2)
        if cykl == 10000-1:
            print ('a: ',str(a_2.T))
            print ('y: ',str(y.T))
        # propagacja "wstecz"
        d_2 = (a_2 - y)*g_prim(a_2)
        d_1 = np.dot(w_2.T, d_2) * g_prim(a_1)#z_2
        
        # akumulujemy poprawki 
        D_2 +=  np.dot( d_2, a_1.T)
        D_1 +=  np.dot( d_1[1:], a_0.T)
        
        bl += np.dot(d_2.T,d_2)
        
    eta1 = 0.6
    # uaktualniamy wagi
    w_1 -=  eta1*D_1 
    w_2 -=  eta1*D_2
    
    # wypisujemy info o błędzie
    if (cykl% 1000) == 0:
        print( 'bl: ', bl)

bl:  [[ 0.10247073]]
bl:  [[ 0.00020618]]
bl:  [[  1.62176569e-05]]
bl:  [[  5.13608978e-06]]
bl:  [[  2.43946393e-06]]
bl:  [[  1.40742773e-06]]
bl:  [[  9.10272313e-07]]
bl:  [[  6.34665251e-07]]
bl:  [[  4.66657908e-07]]
bl:  [[  3.56972890e-07]]
a:  [[ 0.0135618   0.98635568]]
y:  [[0 1]]
a:  [[ 0.98720792  0.01288262]]
y:  [[1 0]]
a:  [[ 0.98720719  0.0128826 ]]
y:  [[1 0]]
a:  [[ 0.01534119  0.98453816]]
y:  [[0 1]]
